In [1]:
import torch

In [2]:
print("CUDA dispo :", torch.cuda.is_available())
print("GPU :", torch.cuda.get_device_name(0))

CUDA dispo : True
GPU : NVIDIA GeForce RTX 2060


# Ok

In [6]:
import os
import shutil
from pathlib import Path

SRC_DIR = Path("clothing-dataset-small-master")
DEST_DIR = Path("ok")

DEST_DIR.mkdir(exist_ok=True)

splits = ["train", "validation", "test"]
extensions = {".jpg", ".jpeg", ".png"}

counter = 1

for split in splits:
    for category in (SRC_DIR / split).iterdir():
        if category.is_dir():
            for img in category.iterdir():
                if img.suffix.lower() in extensions:
                    new_name = f"ok_{counter:06d}{img.suffix.lower()}"
                    shutil.copy2(img, DEST_DIR / new_name)
                    counter += 1

print(f"Terminé ! {counter - 1} images copiées.")


Terminé ! 3781 images copiées.


# Nok

In [7]:
from pathlib import Path
import shutil

# Dossier source : images DeepFashion
SRC_DIR = Path("deepfashion/image")   # adapte si besoin
# Dossier cible : NOK
DEST_DIR = Path("nok")
DEST_DIR.mkdir(exist_ok=True)

# Extensions autorisées
EXTENSIONS = {".jpg", ".jpeg", ".png"}

# Optionnel : si tu veux équilibrer avec le nombre d'OK
OK_DIR = Path("ok")
max_nok = None
if OK_DIR.exists():
    nb_ok = sum(1 for f in OK_DIR.iterdir() if f.suffix.lower() in EXTENSIONS)
    if nb_ok > 0:
        max_nok = nb_ok
        print(f"Dossier 'ok/' détecté : {nb_ok} images. "
              f"On va créer au maximum {max_nok} NOK pour équilibrer.")
    else:
        print("Dossier 'ok/' vide, on copiera toutes les images NOK.")
else:
    print("Pas de dossier 'ok/' détecté, on copiera toutes les images NOK.")

# Récupération des images DeepFashion
all_images = sorted(
    [p for p in SRC_DIR.iterdir() if p.is_file() and p.suffix.lower() in EXTENSIONS]
)

counter = 1
for img in all_images:
    if max_nok is not None and counter > max_nok:
        break

    new_name = f"nok_{counter:06d}{img.suffix.lower()}"
    dest_path = DEST_DIR / new_name
    shutil.copy2(img, dest_path)
    counter += 1

print(f"Terminé. {counter - 1} images NOK copiées dans {DEST_DIR}.")


Dossier 'ok/' détecté : 3781 images. On va créer au maximum 3781 NOK pour équilibrer.
Terminé. 3781 images NOK copiées dans nok.


# Split Train/Test/Val

In [8]:
from pathlib import Path
import shutil
import random

# Dossiers source
OK_DIR = Path("ok")
NOK_DIR = Path("nok")

# Dossier de sortie
OUT_ROOT = Path("dataset")

# Ratios
RATIOS = {
    "train": 0.70,
    "val": 0.15,
    "test": 0.15,
}

# Extensions valides
EXTENSIONS = {".jpg", ".jpeg", ".png"}

random.seed(42)  # reproductibilité


def list_images(folder: Path):
    return sorted(
        [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in EXTENSIONS]
    )


def stratified_split(files, ratios):
    n = len(files)
    n_train = int(n * ratios["train"])
    n_val = int(n * ratios["val"])
    n_test = n - n_train - n_val

    random.shuffle(files)

    return {
        "train": files[:n_train],
        "val": files[n_train : n_train + n_val],
        "test": files[n_train + n_val :],
    }


# Création du dataset
OUT_ROOT.mkdir(exist_ok=True)

data = {
    "ok": list_images(OK_DIR),
    "nok": list_images(NOK_DIR),
}

print("Nombre d'images :")
for label, files in data.items():
    print(f"  {label}: {len(files)} images")

print("\nCréation des splits...")

for label, files in data.items():
    splits = stratified_split(files, RATIOS)

    for split_name, split_list in splits.items():
        dest_dir = OUT_ROOT / split_name / label
        dest_dir.mkdir(parents=True, exist_ok=True)

        for src in split_list:
            dst = dest_dir / src.name
            shutil.copy2(src, dst)

        print(f"{label} → {split_name}: {len(split_list)} images")

print("\nDataset final créé dans dossier 'dataset/'")

Nombre d'images :
  ok: 3781 images
  nok: 3781 images

Création des splits...
ok → train: 2646 images
ok → val: 567 images
ok → test: 568 images
nok → train: 2646 images
nok → val: 567 images
nok → test: 568 images

Dataset final créé dans dossier 'dataset/'
